In [1]:
import os
import sys
# 如果当前代码文件运行测试需要加入修改路径，避免出现后导包问题
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.join(BASE_DIR))
print(BASE_DIR)
PYSPARK_PYTHON = "/miniconda2/envs/reco_sys/bin/python"
# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON
from offline import SparkSessionBase

/root/workspace/toutiao_project/reco_sys


In [2]:
class OriginArticleData(SparkSessionBase):
    
    SPARK_APP_NAME = "mergeArticle"
    SPARK_URL = "local[2]"

    ENABLE_HIVE_SUPPORT = True
    
    def __init__(self):
        self.spark = self._create_spark_session()

In [3]:
oa = OriginArticleData()

In [4]:
# 进行文章 前两个表 的合并
oa.spark.sql("use toutiao")
# news_article_basic 与news_article_content, article_id
titlce_content = oa.spark.sql("select a.article_id, a.channel_id, a.title, b.content from news_article_basic a inner join news_article_content b on a.article_id=b.article_id where a.article_id=116636")

In [5]:
titlce_content.show()

+----------+----------+---------------+--------------------+
|article_id|channel_id|          title|             content|
+----------+----------+---------------+--------------------+
|    116636|        18|动态再平衡投资策略历史数据回测|<p>赚钱是个俗气的话题，但又是人...|
+----------+----------+---------------+--------------------+



In [6]:
# 进行title_content 与 文章频道名称合并
titlce_content.registerTempTable('temptable')

channel_title_content = oa.spark.sql("select t.*, n.channel_name from temptable t left join news_channel n on t.channel_id=n.channel_id")

In [7]:
channel_title_content.show()

+----------+----------+---------------+--------------------+------------+
|article_id|channel_id|          title|             content|channel_name|
+----------+----------+---------------+--------------------+------------+
|    116636|        18|动态再平衡投资策略历史数据回测|<p>赚钱是个俗气的话题，但又是人...|      python|
+----------+----------+---------------+--------------------+------------+



In [8]:
# 合并三个字段内容到一个字符串
import pyspark.sql.functions as F

sentence_df = channel_title_content.select("article_id", "channel_id", "channel_name", "title", "content", 
                            F.concat_ws(',', 
                                       channel_title_content.channel_name,
                                       channel_title_content.title,
                                       channel_title_content.content).alias('sentence'))

In [9]:
sentence_df.show()

+----------+----------+------------+---------------+--------------------+--------------------+
|article_id|channel_id|channel_name|          title|             content|            sentence|
+----------+----------+------------+---------------+--------------------+--------------------+
|    116636|        18|      python|动态再平衡投资策略历史数据回测|<p>赚钱是个俗气的话题，但又是人...|python,动态再平衡投资策略历...|
+----------+----------+------------+---------------+--------------------+--------------------+



In [10]:
# 读取文章，进行每篇文章的分词处理
oa.spark.sql("use article")
article_data = oa.spark.sql("select * from article_data limit 10")
article_data.show()

+----------+----------+------------+--------------------+--------------------+--------------------+
|article_id|channel_id|channel_name|               title|             content|            sentence|
+----------+----------+------------+--------------------+--------------------+--------------------+
|         1|        17|          前端|     Vue props用法小结原荐|<p><strong>Vue pr...|前端,Vue props用法小结原...|
|         2|        17|          前端|vue.js响应式原理解析与实现—...|<p>上次我们已经分析了vue.j...|前端,vue.js响应式原理解析与...|
|         3|        17|          前端|JavaScript中浅拷贝和深拷...|<p>要理解 JavaScript...|前端,JavaScript中浅拷贝...|
|         4|        17|          前端|基于vue2.0 +vuex+ e...|<p>效果演示地址,</p><p>...|前端,基于vue2.0 +vuex...|
|         5|        17|          前端|immutability因Reac...|<p><img src="http...|前端,immutability因R...|
|         6|        17|          前端|简单了解 node npm cnp...|<span id="OSC_h1_...|前端,简单了解 node npm ...|
|         7|        17|          前端|       Web工程师以太坊入门原荐|<p>我经常构建使用以太坊的Web...|前端,Web工程师以太坊入门原荐,...|


In [11]:
# 文章数据进行分词处理,得到分词结果
# 分词
def segmentation(partition):
    import os
    import re

    import jieba
    import jieba.analyse
    import jieba.posseg as pseg
    import codecs

    abspath = "/root/words"

    # 结巴加载用户词典
    userDict_path = os.path.join(abspath, "ITKeywords.txt")
    jieba.load_userdict(userDict_path)

    # 停用词文本
    stopwords_path = os.path.join(abspath, "stopwords.txt")

    def get_stopwords_list():
        """返回stopwords列表"""
        stopwords_list = [i.strip()
                          for i in codecs.open(stopwords_path).readlines()]
        return stopwords_list

    # 所有的停用词列表
    stopwords_list = get_stopwords_list()

    # 分词
    def cut_sentence(sentence):
        """对切割之后的词语进行过滤，去除停用词，保留名词，英文和自定义词库中的词，长度大于2的词"""
        # print(sentence,"*"*100)
        # eg:[pair('今天', 't'), pair('有', 'd'), pair('雾', 'n'), pair('霾', 'g')]
        seg_list = pseg.lcut(sentence)
#         seg_list = [i for i in seg_list if i.flag not in stopwords_list]
        seg_list = [i for i in seg_list if i.word not in stopwords_list]
        filtered_words_list = []
        for seg in seg_list:
            # print(seg)
            if len(seg.word) <= 1:
                continue
            elif seg.flag == "eng":
                if len(seg.word) <= 2:
                    continue
                else:
                    filtered_words_list.append(seg.word)
            elif seg.flag.startswith("n"):
                filtered_words_list.append(seg.word)
            elif seg.flag in ["x", "eng"]:  # 是自定一个词语或者是英文单词
                filtered_words_list.append(seg.word)
        return filtered_words_list

    for row in partition:
        sentence = re.sub("<.*?>", "", row.sentence)    # 替换掉标签数据
        words = cut_sentence(sentence)
        yield row.article_id, row.channel_id, words

In [12]:
words_df = article_data.rdd.mapPartitions(segmentation).toDF(['article_id', 'channel_id', 'words'])

In [13]:
words_df.show()

+----------+----------+--------------------+
|article_id|channel_id|               words|
+----------+----------+--------------------+
|         1|        17|[Vue, props, 用法, ...|
|         2|        17|[vue, 响应式, 原理, mo...|
|         3|        17|[JavaScript, 浅拷贝,...|
|         4|        17|[vue2, vuex, elem...|
|         5|        17|[immutability, Re...|
|         6|        17|[node, npm, cnpm,...|
|         7|        17|[Web, 工程师, 以太坊, 入...|
|         8|        17|[Web, pa, api, we...|
|         9|        17|[vue, 中用, 数据驱动, 视...|
|        10|        17|[程序, WebSocket, 长...|
+----------+----------+--------------------+



In [14]:
# 先计算分词之后的每篇文章的词频，得到CV模型
# 统计所有文章不同的词，组成一个词列表 words_list = [1,2,3,,34,4,45,56,67,78,8.......,,,,.]
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol='words', outputCol='countFeatures', vocabSize=2000, minDF=1.0)
cv_model = cv.fit(words_df)

# 然后根据词频计算IDF以及词，得到IDF模型

In [15]:
cv_model.write().overwrite().save("hdfs://hadoop-master:9000/headlines/models/testCV.model")

In [16]:
from pyspark.ml.feature import CountVectorizerModel
cv_m = CountVectorizerModel.load("hdfs://hadoop-master:9000/headlines/models/testCV.model")

In [17]:
cv_result = cv_m.transform(words_df)

In [18]:
cv_result.show()

+----------+----------+--------------------+--------------------+
|article_id|channel_id|               words|       countFeatures|
+----------+----------+--------------------+--------------------+
|         1|        17|[Vue, props, 用法, ...|(1207,[0,2,3,6,8,...|
|         2|        17|[vue, 响应式, 原理, mo...|(1207,[0,1,2,3,4,...|
|         3|        17|[JavaScript, 浅拷贝,...|(1207,[0,5,10,12,...|
|         4|        17|[vue2, vuex, elem...|(1207,[0,1,3,7,10...|
|         5|        17|[immutability, Re...|(1207,[0,2,3,4,5,...|
|         6|        17|[node, npm, cnpm,...|(1207,[0,1,7,10,1...|
|         7|        17|[Web, 工程师, 以太坊, 入...|(1207,[0,1,2,3,4,...|
|         8|        17|[Web, pa, api, we...|(1207,[0,1,7,26,2...|
|         9|        17|[vue, 中用, 数据驱动, 视...|(1207,[0,3,6,12,1...|
|        10|        17|[程序, WebSocket, 长...|(1207,[0,2,6,10,1...|
+----------+----------+--------------------+--------------------+



In [19]:
# IDF 模型
from pyspark.ml.feature import IDF
idf = IDF(inputCol="countFeatures", outputCol="idfFeatures")
idfModel = idf.fit(cv_result)
idfModel.write().overwrite().save("hdfs://hadoop-master:9000/headlines/models/testIDF.model")

In [20]:
# 可以进行转换
cv_m.vocabulary

['pa',
 'node',
 'data',
 '数据',
 'keys',
 'obj',
 '组件',
 'npm',
 'child',
 '节点',
 'log',
 '属性',
 'key',
 'console',
 'var',
 'return',
 'div',
 '文本',
 'const',
 'fragment',
 '&#',
 'WebSocket',
 'props',
 '合约',
 '对象',
 'msg',
 '代码',
 'ul',
 '.a',
 '程序',
 'model',
 'web3',
 'obj2',
 '以太坊',
 'forChildMsg',
 '交易',
 'class',
 'update',
 'mongodb',
 'config',
 '函数',
 '数组',
 '版本',
 'amp',
 'web',
 '内容',
 'vue',
 '语法',
 'test',
 'childNode',
 'val',
 'ownChildMsg',
 '方法',
 '项目',
 'immutable',
 'match',
 'matchs',
 'function',
 'array',
 'template',
 '信息',
 'Object',
 'DOM',
 '域名',
 'children',
 'Watcher',
 'set',
 '文章',
 '区块链',
 'input',
 'attr',
 '官方',
 '元素节点',
 '方式',
 '命令',
 '功能',
 'install',
 '文件',
 'Array',
 'gas',
 'truffle',
 'newVal',
 'textReg',
 '定义',
 'rawTextContent',
 'item',
 'ckage',
 'registry',
 'Web',
 'arr',
 'JSON',
 'forEach',
 'textContent',
 '智能',
 '类型',
 '镜像',
 '部署',
 '源码',
 'r2',
 'init',
 'rse',
 '服务器',
 'http',
 'mongod',
 'col',
 '地址',
 'call',
 'MVVM',
 'style',
 '

In [21]:
idfModel.idf.toArray()[:20]

array([0.        , 0.6061358 , 0.6061358 , 0.45198512, 1.01160091,
       1.01160091, 1.01160091, 0.78845736, 1.29928298, 1.70474809,
       0.31845373, 1.01160091, 0.78845736, 0.45198512, 0.78845736,
       0.45198512, 1.70474809, 1.29928298, 0.78845736, 1.70474809])

In [22]:
# IDF对CV结果进行计算TFIDF
from pyspark.ml.feature import IDFModel
idf_model = IDFModel.load("hdfs://hadoop-master:9000/headlines/models/testIDF.model")
tfidf_res = idf_model.transform(cv_result)

In [23]:
tfidf_res.show()

+----------+----------+--------------------+--------------------+--------------------+
|article_id|channel_id|               words|       countFeatures|         idfFeatures|
+----------+----------+--------------------+--------------------+--------------------+
|         1|        17|[Vue, props, 用法, ...|(1207,[0,2,3,6,8,...|(1207,[0,2,3,6,8,...|
|         2|        17|[vue, 响应式, 原理, mo...|(1207,[0,1,2,3,4,...|(1207,[0,1,2,3,4,...|
|         3|        17|[JavaScript, 浅拷贝,...|(1207,[0,5,10,12,...|(1207,[0,5,10,12,...|
|         4|        17|[vue2, vuex, elem...|(1207,[0,1,3,7,10...|(1207,[0,1,3,7,10...|
|         5|        17|[immutability, Re...|(1207,[0,2,3,4,5,...|(1207,[0,2,3,4,5,...|
|         6|        17|[node, npm, cnpm,...|(1207,[0,1,7,10,1...|(1207,[0,1,7,10,1...|
|         7|        17|[Web, 工程师, 以太坊, 入...|(1207,[0,1,2,3,4,...|(1207,[0,1,2,3,4,...|
|         8|        17|[Web, pa, api, we...|(1207,[0,1,7,26,2...|(1207,[0,1,7,26,2...|
|         9|        17|[vue, 中用, 数据驱动, 视...

In [24]:
# 1265词的 {索引 以及 权重}
def func(partition):
    TOPK = 20
    for row in partition:
        # 找到索引与IDF值并进行排序
        _ = list(zip(row.idfFeatures.indices, row.idfFeatures.values))
        _ = sorted(_, key=lambda x: x[1], reverse=True)
        result = _[:TOPK]
        for word_index, tfidf in result:
            yield row.article_id, row.channel_id, int(word_index), round(float(tfidf), 4)
kewords_tfidf = tfidf_res.rdd.mapPartitions(func).toDF(['article_id', 'channel_id', 'index', 'weights'])

In [25]:
kewords_tfidf.show()

+----------+----------+-----+-------+
|article_id|channel_id|index|weights|
+----------+----------+-----+-------+
|         1|        17|   16|68.1899|
|         1|        17|    8|58.4677|
|         1|        17|    6|55.6381|
|         1|        17|   22|52.8472|
|         1|        17|   34|39.2092|
|         1|        17|   25|31.1828|
|         1|        17|   49|30.6855|
|         1|        17|   51|30.6855|
|         1|        17|   59| 27.276|
|         1|        17|  127|15.3427|
|         1|        17|  128|15.3427|
|         1|        17|  136| 13.638|
|         1|        17|  156| 13.638|
|         1|        17|  159| 13.638|
|         1|        17|    3|11.2996|
|         1|        17|  191|10.2285|
|         1|        17|  225| 8.5237|
|         1|        17|  263| 8.5237|
|         1|        17|  278|  6.819|
|         1|        17|  286|  6.819|
+----------+----------+-----+-------+
only showing top 20 rows



In [26]:
# 利用keywordsIndex = ktt.spark.sql("select keyword, index idx from idf_keywords_values")中标，知道索引对应的词
idf_keywords_values = oa.spark.sql("select keyword, index idx from idf_keywords_values")

In [27]:
keyword_str_tfidf = kewords_tfidf.join(idf_keywords_values, idf_keywords_values.idx==kewords_tfidf.index).select(["article_id", "channel_id", "keyword", "weights"])

keyword_str_tfidf.show()

+----------+----------+-------+-------+
|article_id|channel_id|keyword|weights|
+----------+----------+-------+-------+
|         9|        17|    var|  3.503|
|        10|        17|     属性|31.1828|
|         2|        17|  https|25.5712|
|         1|        17|   人工智能|10.2285|
|         4|        17|     脚本|  6.819|
|         1|        17|     视频|  6.819|
|         2|        17|    com|61.3709|
|         5|        17|     功能|28.9807|
|         8|        17|    SQL| 5.1142|
|         5|        17|     图片|10.3943|
|         7|        17|     架构|  9.095|
|        10|        17|     手机| 8.5237|
|         1|        17|     用户|52.8472|
|         6|        17|    web| 7.0812|
|        10|        17| github|10.2285|
|         8|        17|     思路| 5.1142|
|         6|        17|     对象|38.6344|
|         1|        17|    the|39.2092|
|         8|        17|     数字| 5.1971|
|         7|        17|    arr|10.2285|
+----------+----------+-------+-------+
only showing top 20 rows



In [28]:
# texrank
# 分词
def textrank(partition):
    import os

    import jieba
    import jieba.analyse
    import jieba.posseg as pseg
    import codecs

    abspath = "/root/words"

    # 结巴加载用户词典
    userDict_path = os.path.join(abspath, "ITKeywords.txt")
    jieba.load_userdict(userDict_path)

    # 停用词文本
    stopwords_path = os.path.join(abspath, "stopwords.txt")

    def get_stopwords_list():
        """返回stopwords列表"""
        stopwords_list = [i.strip()
                          for i in codecs.open(stopwords_path).readlines()]
        return stopwords_list

    # 所有的停用词列表
    stopwords_list = get_stopwords_list()

    class TextRank(jieba.analyse.TextRank):
        def __init__(self, window=20, word_min_len=2):
            super(TextRank, self).__init__()
            self.span = window  # 窗口大小
            self.word_min_len = word_min_len  # 单词的最小长度
            # 要保留的词性，根据jieba github ，具体参见https://github.com/baidu/lac
            self.pos_filt = frozenset(
                ('n', 'x', 'eng', 'f', 's', 't', 'nr', 'ns', 'nt', "nw", "nz", "PER", "LOC", "ORG"))

        def pairfilter(self, wp):
            """过滤条件，返回True或者False"""

            if wp.flag == "eng":
                if len(wp.word) <= 2:
                    return False

            if wp.flag in self.pos_filt and len(wp.word.strip()) >= self.word_min_len \
                    and wp.word.lower() not in stopwords_list:
                return True
    # TextRank过滤窗口大小为5，单词最小为2
    textrank_model = TextRank(window=5, word_min_len=2)
    allowPOS = ('n', "x", 'eng', 'nr', 'ns', 'nt', "nw", "nz", "c")

    for row in partition:
        tags = textrank_model.textrank(row.sentence, topK=20, withWeight=True, allowPOS=allowPOS, withFlag=False)
        for tag in tags:
            yield row.article_id, row.channel_id, tag[0], tag[1]

In [29]:
textrank = article_data.rdd.mapPartitions(textrank).toDF(["article_id", "channel_id", "keyword", "textrank"])